# Imports and Paths

In [275]:
import urllib
from bs4 import BeautifulSoup, Comment
import os
import pandas as pd
from plotnine import *
import plotnine
import time

plotnine.options.figure_size = (12, 8)
from pandas.api.types import is_string_dtype
pd.options.display.max_columns=100
pd.options.display.max_rows=100

In [296]:
PATH = '/data/msnow/football/'

# Getting Team pages

In [20]:
base_pg = 'https://www.pro-football-reference.com'
year_pg_tmplt = 'https://www.pro-football-reference.com/years/'
# year_pg_tmplt = 'https://www.pro-football-reference.com/years/1966/'

In [245]:
year = 1966
year_pg_url = os.path.join(year_pg_tmplt,str(year))
year_pg = urllib.request.urlopen(year_pg_url)
soup_year = BeautifulSoup(year_pg, 'html.parser')

In [260]:
teams = soup_year.find("table", {'id':'NFL'}).find_all('a')
team_number = 0

# Extracting info

In [351]:
df_dict.keys()

dict_keys(['df_1966_00', 'df_1966_01', 'df_1966_02', 'df_1966_03', 'df_1966_04', 'df_1966_05', 'df_1966_06', 'df_1966_07', 'df_1966_08', 'df_1966_09', 'df_1966_10', 'df_1966_11', 'df_1966_12', 'df_1966_13', 'df_1966_14', 'df_1967_00', 'df_1967_01', 'df_1967_02', 'df_1967_03', 'df_1967_04', 'df_1967_05', 'df_1967_06', 'df_1967_07', 'df_1967_08', 'df_1967_09', 'df_1967_10', 'df_1967_11', 'df_1967_12', 'df_1967_13', 'df_1967_14', 'df_1967_15', 'df_1968_00', 'df_1968_01', 'df_1968_02', 'df_1968_03', 'df_1968_04', 'df_1968_05', 'df_1968_06', 'df_1968_07', 'df_1968_08', 'df_1968_09', 'df_1968_10', 'df_1968_11', 'df_1968_12', 'df_1968_13', 'df_1968_14', 'df_1968_15', 'df_1969_00', 'df_1969_01', 'df_1969_02', 'df_1969_03', 'df_1969_04', 'df_1969_05', 'df_1969_06', 'df_1969_07', 'df_1969_08', 'df_1969_09', 'df_1969_10', 'df_1969_11', 'df_1969_12', 'df_1969_13', 'df_1969_14', 'df_1969_15', 'df_1970_00', 'df_1970_01', 'df_1970_02', 'df_1970_03', 'df_1970_04', 'df_1970_05', 'df_1970_06', 'df_1970_

In [352]:
# df_dict2 = df_dict.copy()
df_teams = pd.concat(df_dict.values(), ignore_index=True)
df_teams.to_feather(f'{PATH}df_teams_1966_2017.feather')

In [330]:
df_some.shape

(9002, 26)

In [344]:
year, x

(1992, 0)

In [350]:
# df_dict = {}
for year in range(1992,2018):
    year_pg_url = os.path.join(year_pg_tmplt,str(year))
    year_pg = urllib.request.urlopen(year_pg_url)
    soup_year = BeautifulSoup(year_pg, 'html.parser')
    if year < 1970:
        teams = soup_year.find("table", {'id':'NFL'}).find_all('a')
    else:
        teams = soup_year.find("table", {'id':'AFC'}).find_all('a') + soup_year.find("table", {'id':'NFC'}).find_all('a')
    for x in range(len(teams)):
        df_dict['df_'+ str(year) + '_' + str(x).zfill(2)] = extract_from_year(base_pg, soup_year, teams, x) 
        time.sleep(1)
df_all = pd.concat(df_dict.values(), ignore_index=True)

In [263]:
def extract_from_year(base_pg, soup_year, teams, team_number):
    team_pg_url = base_pg + teams[team_number].get('href')
    team_pg = urllib.request.urlopen(team_pg_url)
    soup_team = BeautifulSoup(team_pg, 'lxml')
    game_data = extract_from_team(soup_team)
    df = pd.DataFrame.from_dict(game_data)
    return df

In [323]:
def extract_from_team(soup_team):
    game_data = {}
    comments=soup_team.find_all(string=lambda text:isinstance(text,Comment))
    game_data, game_sz = results(game_data, soup_team)
    game_data = kicks(game_data,comments, game_sz)
    game_data = fumbl(game_data,comments, game_sz)
    game_data = other_td(game_data,comments, game_sz)
    game_data = blocked_punts(game_data,comments, game_sz)
    return game_data

## Game results

In [60]:
table = soup_team.find('table', {'id':'games'})
table_rows = table.find_all('tr')

In [272]:
base_info

['\n',
 <span>1966</span>,
 '\n',
 <span>Dallas Cowboys</span>,
 '\n',
 <span class="header_end">Statistics &amp; Players</span>,
 '\n']

In [349]:
def results(game_data, soup_team):
    table = soup_team.find('table', {'id':'games'})
    table_rows = table.find_all('tr')
    game_date = []
    skip_idx = -999
    for idx, val in enumerate(table.find_all('td',{'data-stat':'game_date'})):
        if val.string=='Playoffs':
            break
        try:
            game_date.append(val['csk'])
        except:
            skip_idx = idx
            pass
    game_data['game_date'] = game_date
    game_sz = len(game_date)
    if skip_idx > 0:
        game_sz_games = game_sz + 1
    else:
        game_sz_games = game_sz


    var_list = ['opp','pts_off','pts_def','first_down_off','yards_off','pass_yds_off','rush_yds_off','to_off','first_down_def','yards_def','pass_yds_def','rush_yds_def','to_def']
    num_set = {'pts_off','pts_def','first_down_off','yards_off','pass_yds_off','rush_yds_off','to_off','first_down_def','yards_def','pass_yds_def','rush_yds_def','to_def'}
    for var_name in var_list:
        var_val_list = []
        for idx,val in enumerate(table.find_all('td',{'data-stat':var_name})[:game_sz_games]):
            if idx != skip_idx:
                if val.string is None:
                    var_val_list.append(0)
                elif var_name in num_set:
                    var_val_list.append(int(val.string))
                else:
                    var_val_list.append(val.string)
        game_data[var_name] = var_val_list
    
    team_name = soup_team.find('h1', {'itemprop':'name'})
    base_info = [x for x in team_name.children]
    game_data['team'] = [base_info[3].string]*game_sz
    game_data['Year'] = [int(base_info[1].string)]*game_sz
    return game_data, game_sz

## Kicks and punts

In [171]:
comments=soup_team.find_all(string=lambda text:isinstance(text,Comment))

In [321]:
def kicks(game_data,comments,game_sz):       
    for v in comments:
        if v.find('div_returns') > -1:
            table = BeautifulSoup(str(v), 'lxml')
            break
    for x in table.find('td',text='Team Total').next_siblings:
        if x['data-stat'] == 'punt_ret_td':
            if x.string is None:
                game_data['punt_ret_td'] = [0]*game_sz
            else:
                game_data['punt_ret_td'] = [int(x.string)/game_sz]*game_sz
        if x['data-stat'] == 'kick_ret_td':
            if x.string is None:
                game_data['kick_ret_td'] = [0]*game_sz
            else:
                game_data['kick_ret_td'] = [int(x.string)/game_sz]*game_sz
    return game_data

## Fumbles and sacks

In [320]:
def fumbl(game_data,comments,game_sz):   
    for v in comments:
        if v.find('div_defense') > -1:
            table = BeautifulSoup(str(v), 'lxml')
            break
    tst_dict = {}
    fumbl_set = {'sacks','fumbles_rec_td','safety_md','fumbles_rec','def_int'}
    for x in table.find('td',text='Team Total').next_siblings:
        if x['data-stat'] in fumbl_set:
            if x.string is None:
                game_data[x['data-stat']] = [0]*game_sz
            else:
                game_data[x['data-stat']] = [int(x.string)/game_sz]*game_sz
    return game_data

## Other TDs

In [319]:
def other_td(game_data,comments,game_sz):
    for v in comments:
        if v.find('div_scoring') > -1:
            table = BeautifulSoup(str(v), 'lxml')
            break
    tst_dict = {}
    other_set = {'ditd','otd'}
    other_dict = {'ditd':'int_td','otd':'blocked_fg_td'}
    for x in table.find('td',text='Team Total').next_siblings:
        if x['data-stat'] in other_set:
            if x.string is None:
                game_data[other_dict[x['data-stat']]] = [0]*game_sz
            else:
                game_data[other_dict[x['data-stat']]] = [int(x.string)/game_sz]*game_sz
    return game_data

## Blocked punts

In [318]:
def blocked_punts(game_data,comments,game_sz):
    for v in comments:
        if v.find('div_kicking') > -1:
            table = BeautifulSoup(str(v), 'lxml')
            break
    tst_dict = {}
    blocked_set = {'punt_blocked'}
    for x in table.find('td',text='Opp Total').next_siblings:
        if x['data-stat'] in blocked_set:
            if x.string is None:
                game_data[x['data-stat']] = [0]*game_sz
            else:
                game_data[x['data-stat']] = [int(x.string)/game_sz]*game_sz
    return game_data

# DataFrame

In [219]:
df = pd.DataFrame.from_dict(game_data)
df

,Year,blocked_fg_td,def_int,first_down_def,first_down_off,fumbles_rec,fumbles_rec_td,game_date,int_td,kick_ret_td,...,punt_ret_td,rush_yds_def,rush_yds_off,sacks,safety_md,team,to_def,to_off,yards_def,yards_off
0,1966,0,1.214286,18,24,1.928571,0.071429,1966-09-18,0.214286,0.071429,...,0.0,137,144,4.285714,0.071429,Dallas Cowboys,4,1,257,518
1,1966,0,1.214286,25,18,1.928571,0.071429,1966-09-25,0.214286,0.071429,...,0.0,142,148,4.285714,0.071429,Dallas Cowboys,1,0,389,314
2,1966,0,1.214286,20,22,1.928571,0.071429,1966-10-02,0.214286,0.071429,...,0.0,106,143,4.285714,0.071429,Dallas Cowboys,5,2,276,363
3,1966,0,1.214286,11,32,1.928571,0.071429,1966-10-09,0.214286,0.071429,...,0.0,38,212,4.285714,0.071429,Dallas Cowboys,1,0,217,652
4,1966,0,1.214286,13,17,1.928571,0.071429,1966-10-16,0.214286,0.071429,...,0.0,45,135,4.285714,0.071429,Dallas Cowboys,0,4,175,326
5,1966,0,1.214286,17,30,1.928571,0.071429,1966-10-23,0.214286,0.071429,...,0.0,105,156,4.285714,0.071429,Dallas Cowboys,0,4,264,413
6,1966,0,1.214286,6,24,1.928571,0.071429,1966-10-30,0.214286,0.071429,...,0.0,7,192,4.285714,0.071429,Dallas Cowboys,4,3,119,425
7,1966,0,1.214286,5,16,1.928571,0.071429,1966-11-06,0.214286,0.071429,...,0.0,58,112,4.285714,0.071429,Dallas Cowboys,3,2,80,240
8,1966,0,1.214286,19,24,1.928571,0.071429,1966-11-13,0.214286,0.071429,...,0.0,23,132,4.285714,0.071429,Dallas Cowboys,0,3,341,515
9,1966,0,1.214286,16,16,1.928571,0.071429,1966-11-20,0.214286,0.071429,...,0.0,53,124,4.285714,0.071429,Dallas Cowboys,3,1,163,258
